In [8]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageTk#, ImageFilter
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import jdatetime
from datetime import datetime, timedelta
import time

In [9]:
# Constants
IMAGE_FOLDER = r"d:\slideshow"
logo         = r"d:\logo.png"

In [10]:
# Function to scrape data from https://www.estjt.ir/price/
def scrape_tala_ir():
    url = "https://www.estjt.ir/price/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup)
    # Locate the table
    # Function to extract data from a table
    def extract_table_data(table):
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')
        data = []
        for row in rows:
            cells = row.find_all('td')
            name = cells[0].text.strip()  # First cell is the name
            current_price = cells[1].text.strip()  # Second cell is the current price
            data.append([name, current_price])
        return headers[:2], data  # Only keep the first two headers

    # Extract gold prices
    gold_table = soup.find('h2', text='قیمت طلا و آخرین تغییرات').find_next('table')
    gold_headers, gold_data = extract_table_data(gold_table)

    # Extract coin prices
    coin_table = soup.find('h2', text='قیمت سکه و آخرین تغییرات').find_next('table')
    coin_headers, coin_data = extract_table_data(coin_table)

    # Combine headers and data
    all_headers = gold_headers  # Both tables have the same headers
    all_data = gold_data + coin_data  # Combine gold and coin data

    # Create a DataFrame
    df = pd.DataFrame(all_data, columns=all_headers)
    print(df.columns)
    df = df[['نرخ فعلی', 'نوع طلا']]
    df.columns = [['قیمت','عنوان']]
    return df



In [11]:
# Function to update the clock and date
def update_clock():
    # Get current time
    current_time = time.strftime("%H:%M:%S")
    # Get Jalali date
    jalali_date = jdatetime.datetime.now().strftime("%Y/%m/%d")
    # Update the clock and date label
    clock_date_label.config(text=f"تاریخ: {jalali_date}\nزمان: {current_time}")
    # Update every second
    root.after(1000, update_clock)

In [12]:
# Function to load images for the slideshow
def load_images(folder):
    images = []
    image_paths = []    
    for filename in os.listdir(folder):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path)            
            try:
                # For Pillow 10.0.0 and newer
                resized_image = img.resize(((int(root.winfo_screenwidth()*.75)), root.winfo_screenheight()), Image.Resampling.LANCZOS)
            except AttributeError:
                # For older versions of Pillow
                resized_image = img.resize(((int(root.winfo_screenwidth()*.75)), root.winfo_screenheight()), Image.ANTIALIAS)
            rgb_image = resized_image.convert("RGB")
            rgb_image.save(f"{filename}.jpg", "JPEG")
            images.append(ImageTk.PhotoImage(rgb_image))
            image_paths.append(img_path)
    return images, image_paths 

In [13]:
# Function to update the slideshow with random transitions
def update_slideshow(index):
    if images:
        # Get the next image
        next_index = (index + 1) % len(images)  # Wrap around to 0 at the end
        next_image = images[next_index]

        # Update the displayed image
        image_label.config(image=next_image)
        image_label.image = next_image  # Keep a reference to avoid garbage collection

        # Schedule the next slideshow update
        root.after(10000, update_slideshow, next_index)  # 3000ms = 3 seconds delay

In [15]:
# Function to refresh the page
def refresh_page():
    scraped_data = scrape_tala_ir()
    data_label.config(text=scraped_data.to_string(index=False) + '\n به روزرسانی شده در ' + time.strftime("%H:%M:%S") )
    root.after(10000, refresh_page)  # Refresh every 10 secs

In [16]:
# Initialize the main window
root = tk.Tk()
root.title("اطلاعات بازار")
root.geometry("1366x768")

# Create a frame for the left side (75% of the window)
left_frame = tk.Frame(root, width=int(root.winfo_screenwidth() * 0.75), height=root.winfo_screenheight())
left_frame.pack(side="left", fill="both", expand=True,padx=0, pady=0)

# Add a label for the slideshow
image_label = tk.Label(left_frame)
image_label.pack(fill="both", expand=True)

# Load images and start the slideshow
images, image_paths = load_images(IMAGE_FOLDER)  # Load images and their paths
update_slideshow(0)

In [17]:
# Create a frame for the right side (25% of the window)
right_frame = tk.Frame(root, width=int(root.winfo_screenwidth() * 0.25), height=root.winfo_screenheight())
right_frame.pack(side="right", fill="both", expand=True, padx=0, pady=0)


logo_image = Image.open(logo)  # Replace with the path to your logo
try:
    # For Pillow 10.0.0 and newer
    logo_image = logo_image.resize((int(root.winfo_screenwidth()*.25), int(root.winfo_screenheight()*.25) ), Image.Resampling.LANCZOS)
except AttributeError:
    # For older versions of Pillow
    logo_image = logo_image.resize((int(root.winfo_screenwidth()*.25), int(root.winfo_screenheight()*.25) ), Image.ANTIALIAS)
logo_photo = ImageTk.PhotoImage(logo_image)
logo_label = tk.Label(right_frame, image=logo_photo)
logo_label.pack(padx=10, side="top", pady=10)

# Add a label for scraped data (table-like format)
data_label = tk.Label(right_frame, text="tala.ir", font=("b nazanin", 18), bg="lightblue", fg="black", bd=2, relief="solid", justify="right")
data_label.pack( padx=10, pady=10, fill="x")


# Add a label for the digital clock and Jalali date
clock_date_label = tk.Label(right_frame, font=("Arial", 14), bg="lightgreen", fg="black", bd=2, relief="solid")
clock_date_label.pack(padx=10, side="bottom", pady=10, fill="x")

In [18]:
# Start the clock
update_clock() # 1 second

# Scrape data from tala.ir and refresh the page every 1 minutes
refresh_page()

# Run the application
root.mainloop()

C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  gold_table = soup.find('h2', text='قیمت طلا و آخرین تغییرات').find_next('table')
C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  coin_table = soup.find('h2', text='قیمت سکه و آخرین تغییرات').find_next('table')


Index(['نوع طلا', 'نرخ فعلی'], dtype='object')


C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  gold_table = soup.find('h2', text='قیمت طلا و آخرین تغییرات').find_next('table')
C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  coin_table = soup.find('h2', text='قیمت سکه و آخرین تغییرات').find_next('table')


Index(['نوع طلا', 'نرخ فعلی'], dtype='object')


C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  gold_table = soup.find('h2', text='قیمت طلا و آخرین تغییرات').find_next('table')
C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  coin_table = soup.find('h2', text='قیمت سکه و آخرین تغییرات').find_next('table')


Index(['نوع طلا', 'نرخ فعلی'], dtype='object')


C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:21: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  gold_table = soup.find('h2', text='قیمت طلا و آخرین تغییرات').find_next('table')
C:\Users\mr.rahimy\AppData\Local\Temp\ipykernel_7656\810323292.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  coin_table = soup.find('h2', text='قیمت سکه و آخرین تغییرات').find_next('table')


Index(['نوع طلا', 'نرخ فعلی'], dtype='object')


In [11]:
# pyinstaller --onefile --windowed your_script_name.py